In [1]:
%pip install git+https://github.com/JustAnotherArchivist/snscrape.git

  Cloning https://github.com/JustAnotherArchivist/snscrape.git to c:\users\ade\appdata\local\temp\pip-req-build-2w8l060s
  Resolved https://github.com/JustAnotherArchivist/snscrape.git to commit 786815dd05681e2421cd03aa9acf5ab5c773bce9
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ---------------------------------------- 3.8/3.8 MB 975.2 kB/s eta 0:00:00
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
     ------------------------------------ 143.0/143.0 kB 850.9 kB/s eta 0:00:00
     ---------------------------------------- 62.6/62.6 kB 1.7 MB/s eta 0:00:00
  Using cached idna-3.4-py3-none

  Running command git clone --filter=blob:none --quiet https://github.com/JustAnotherArchivist/snscrape.git 'C:\Users\Ade\AppData\Local\Temp\pip-req-build-2w8l060s'

[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
%pip install pandas

     ---------------------------------------- 10.7/10.7 MB 2.1 MB/s eta 0:00:00
  Using cached tzdata-2023.3-py2.py3-none-any.whl (341 kB)
     ---------------------------------------- 14.8/14.8 MB 1.5 MB/s eta 0:00:00
     -------------------------------------- 502.3/502.3 kB 2.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
import sqlite3
import warnings


In [38]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import warnings

# Ignorar las advertencias de DeprecatedFeatureWarning
#warnings.filterwarnings("ignore", category=sntwitter.DeprecatedFeatureWarning)

# Definir la cuenta de Twitter mencionada y el rango de fechas
account = "@TheBridge_Tech"
start_date = "2023-02-13"
end_date = "2023-05-22"

# Construir la consulta de búsqueda
query = f"{account} since:{start_date} until:{end_date}"

# Obtener los tweets utilizando snscrape
tweets = []
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    tweet_id = tweet.id
    text = tweet.rawContent
    date = tweet.date.strftime('%Y-%m-%d %H:%M:%S')
    author_id = tweet.user.id
    author_name = tweet.user.name if hasattr(tweet.user, 'name') else None
    author_username = tweet.user.username
    retweets = tweet.retweetCount
    replies = tweet.replyCount
    likes = tweet.likeCount
    quotes = tweet.quoteCount

    tweets.append([tweet_id, text, date, author_id, author_name, author_username, retweets, replies, likes, quotes])

    # Romper el bucle si se alcanza el límite de tweets deseados
    if len(tweets) >= 10000:
        break

# Crear un DataFrame a partir de la lista de tweets
columns = ["ID", "Text", "Date", "Author ID", "Author Name", "Author Username", "Retweets", "Replies", "Likes", "Quotes"]
tweets_df = pd.DataFrame(tweets, columns=columns)

# Imprimir el DataFrame
print(tweets_df)

Stopping after 20 empty pages


                      ID                                               Text   
0    1659514560200470559  Ayer visitamos a los alumnos de @TheBridge_Tec...  \
1    1659136842854219776  ¿Qué hacían nuestros alumni antes del #bootcam...   
2    1658825158994403329  #MLMonitoring and #Observability, mar, 23 may ...   
3    1658791064306688002  Nos colamos en el campus de #Bilbao para descu...   
4    1658430987368386566  Así es el #bootcamp de #UXUIProductDesign que ...   
..                   ...                                                ...   
138  1625828554691805187  ¿Los colores tienen género?¿podemos hacer una ...   
139  1625825721862987777                       @BitcoinTuesday_ ¡Un placer!   
140  1625813763751305216  Gracias a nuestros amigos @TheBridge_Tech por ...   
141  1625165984414695426  ¡Nos vamos al WorkinLan Workshop en Bilbao jun...   
142  1625126689150738438  Les compartimos un mini resumen del 1º encuent...   

                    Date            Author ID Autho

In [39]:
tweets_df.describe()

,ID,Author ID,Retweets,Replies,Likes,Quotes
count,1.430000e+02,1.430000e+02,143.000000,143.000000,143.000000,143.000000
mean,1.640923e+18,7.433382e+17,1.468531,0.349650,3.503497,0.153846
std,9.297728e+15,5.770250e+17,2.666443,0.674054,4.874299,0.521478
min,1.625127e+18,7.154412e+06,0.000000,0.000000,0.000000,0.000000
25%,1.634147e+18,1.003872e+09,0.000000,0.000000,1.000000,0.000000
50%,1.638579e+18,9.365398e+17,1.000000,0.000000,2.000000,0.000000
75%,1.648301e+18,1.162694e+18,2.000000,1.000000,4.000000,0.000000
max,1.659515e+18,1.613114e+18,25.000000,4.000000,39.000000,4.000000


In [40]:
# Obtener el usuario con la mayor cantidad de tweets
user_with_most_tweets = tweets_df['Author Username'].value_counts().idxmax()

# Imprimir el resultado
print("El usuario con la mayor cantidad de tweets es:", user_with_most_tweets)


El usuario con la mayor cantidad de tweets es: TheBridge_Tech


In [41]:
# Obtener todos los usuarios con la mayor cantidad de tweets
users_with_most_tweets = tweets_df['Author Username'].value_counts()
max_tweets = users_with_most_tweets.max()
users_with_max_tweets = users_with_most_tweets[users_with_most_tweets == max_tweets]

# Imprimir los usuarios con la mayor cantidad de tweets
print("Usuarios con la mayor cantidad de tweets:")
for user in users_with_max_tweets.index:
    print(user)


Usuarios con la mayor cantidad de tweets:
TheBridge_Tech


In [42]:
# Obtener los 10 usuarios con la mayor cantidad de tweets
top_10_users = tweets_df['Author Username'].value_counts().nlargest(10)

# Imprimir los 10 usuarios con la mayor cantidad de tweets
print("Los 10 usuarios con la mayor cantidad de tweets:")
print(top_10_users)


Los 10 usuarios con la mayor cantidad de tweets:
Author Username
TheBridge_Tech     45
DatabeerSVQ        17
nodosenlared        9
ESOLIUPO            6
decentralized_b     5
ToniDorta           4
espacio_RES         4
hello_google        4
PauMugarra          4
turoninos           3
Name: count, dtype: int64


In [47]:
top_10_users = tweets_df['Author Username'].value_counts().nlargest(10).index.tolist()
top_10_df = tweets_df[tweets_df['Author Username'].isin(top_10_users)]
top_retweet_users = tweets_df.groupby('Author Username')['Retweets'].sum().nlargest(10).index.tolist()
top_retweet_df = tweets_df[tweets_df['Author Username'].isin(top_retweet_users)]
top_tweet_ids = tweets_df[['Retweets', 'Replies', 'Likes', 'Quotes']].sum(axis=1).nlargest(10).index.tolist()
top_tweet_df = tweets_df.loc[top_tweet_ids]
tweet_best = pd.concat([top_10_df, top_retweet_df, top_tweet_df])
tweet_best


,ID,Text,Date,Author ID,Author Name,Author Username,Retweets,Replies,Likes,Quotes
1,1659136842854219776,¿Qué hacían nuestros alumni antes del #bootcam...,2023-05-18 10:00:23,1162694149956603904,None,TheBridge_Tech,0,0,0,0
2,1658825158994403329,"#MLMonitoring and #Observability, mar, 23 may ...",2023-05-17 13:21:52,41545659,None,nodosenlared,1,0,3,0
3,1658791064306688002,Nos colamos en el campus de #Bilbao para descu...,2023-05-17 11:06:23,1162694149956603904,None,TheBridge_Tech,1,0,2,0
4,1658430987368386566,Así es el #bootcamp de #UXUIProductDesign que ...,2023-05-16 11:15:34,1162694149956603904,None,TheBridge_Tech,1,0,3,0
5,1658211678901641230,"Y finalmente, como nos decían en casa, es de b...",2023-05-15 20:44:06,1585726199485140993,None,ESOLIUPO,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
83,1636418703528017945,¡Presentamos cartelón para #DatabeerSVQ19! Nos...,2023-03-16 17:26:36,936539844079218688,None,DatabeerSVQ,7,0,10,0
91,1635802357568253952,Gran meetup en Madrid de @BitcoinTuesday_\n\nG...,2023-03-15 00:37:28,1207228796593745920,None,decentralized_b,0,0,15,1
125,1631600898777444354,Ya esta publicado el meetup de @BitcoinTuesday...,2023-03-03 10:22:22,1207228796593745920,None,decentralized_b,4,0,11,1
134,1626267048434192385,".@EstiLeon, Responsable del campus #42UrdulizF...",2023-02-16 17:07:33,1288769305014939650,None,42UrdulizFTef,5,1,10,0
